In [1]:
import matplotlib
%matplotlib
import math
import cmath
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc
from scipy import ndimage
from numpy.linalg import inv
from scipy.linalg import circulant
import cv2

Using matplotlib backend: Qt5Agg


In [2]:
#Function to estimate the variance of the supplied window sized image using histogram of the patch
def estimate_var(img,M1,N1,M2,N2):
    #Taking window of the defined size
    img1 = img[M1:M2,N1:N2]
    plt.figure()
    n , bins , patches = plt.hist(img1.ravel(), bins=256, range=(0, 255),normed='True')
    plt.show()    
    mean = 0
    for i in range(n.shape[0]):
        mean += i * n[i]
    img1 = img1 - mean
    var = 0
    for i in range(n.shape[0]):
        var += ((i-mean) ** 2) * n[i]
    return var
    

In [3]:
#Function to 
def myAdaptiveLNR(img,M1,N1,M2,N2,window_size):
    var_n = estimate_var(img,M1,N1,M2,N2)
    print(var_n)
    M = img.shape[0]
    N = img.shape[1]
    k = (window_size-1) // 2
    img1 = np.zeros(shape=(M+2*k,N+2*k),dtype=np.unsignedinteger)
    img1[k:M+k,k:N+k] = img
    #plt.imshow(img1)
    for i in range(k,M+k):
        for j in range(k,N+k):
            sub_i = img1[i-k:i+k+1,j-k:j+k+1]
            var_loc = np.var(sub_i)
            mean_loc = np.mean(sub_i)
            if (var_n < var_loc):
                img[i-k,j-k] = int(img1[i-k,j-k] -  (var_n/var_loc) * (img1[i-k,j-k] - mean_loc))
            else:
                img[i-k,j-k] = mean_loc
    return img,var_n

In [4]:
def imnoise(img,noise_type,amt=0):
    if(noise_type == 'gaussian'):
        row,col = img.shape
        mean = 5
        var = 15
        sigma = var**0.5
        gauss_noise = np.random.normal(mean,sigma,(row,col))
        gauss_noise = gauss_noise.reshape(row,col)
        noisy_img = img + gauss_noise
        return noisy_img
    elif noise_type == "s&p":
        row,col= img.shape
        s_vs_p = 0.5
        amount = amt
        out = np.copy(img)
      # Salt mode
        num_salt = np.ceil((amount/2) * img.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
                  for i in img.shape]
        out[coords] = 255

      # Pepper mode
        num_pepper = np.ceil((amount/2)* img.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in img.shape]
        out[coords] = 0
        return out
    else:
        return

In [5]:
def myAdaptiveMed(img,Smax):
    Smin = 3
    count = 0
    M = img.shape[0]
    N = img.shape[1]
    k = (Smax-1) // 2
    img1 = np.zeros(shape=(M+2*k,N+2*k))
    img1[k:M+k,k:N+k] = img
    for i in range(k,M-k):
        for j in range(k,N-k):
            flag = 1
            Smin = 3
            #print("yes")
            while(flag):
                k1 = (Smin - 1) // 2
                #print(i,j,k1)
                sub_i = img1[i-k1:i+k1+1,j-k1:j+k1+1]
                #print(sub_i.shape)
                zmed = int(np.median(sub_i))
                zmax = np.max(sub_i)
                zmin = np.min(sub_i)
                A1 = zmed - zmin
                A2 = zmed - zmax
                if A1 > 0 and A2 < 0:
                    B1 = img1[i,j] - zmin
                    B2 = img1[i,j] - zmax
                    if(B1 > 0 and B2 < 0):
                        img[i,j] = img1[i,j]
                        flag = 0
                    else:
                        flag = 0
                        img[i,j] = zmed
                else:
                    Smin += 2
                    if(Smin > Smax):
                        flag = 0
                        count += 1
                        img[i,j] = img1[i,j]
                        break
    return img

In [6]:
def mymed2d(img,M,N):
    new_img = np.zeros(shape=(img.shape[0]+M,img.shape[1] + N),dtype=np.integer)
    new_img[M-1 : -1,N-1 : -1] = img
    new_med_img = np.zeros(shape=(img.shape[0],img.shape[1] ))
    for i in range(new_med_img.shape[0]):
        for j in range(new_med_img.shape[1]):
            new_med_img[i,j] = int(np.median(new_img[i:i+M , j:j+N]))                
    return new_med_img

In [9]:
img = cv2.imread('noisypeppers.png',0)
print("Original Image")
plt.figure()
plt.imshow(img)
plt.gray()
'''
w_si
density = [0.05,0.1,0.5]
for dens in density:
    noisy_img  = imnoise(img,'s&p',dens)

    output = myAdaptiveMed(noisy_img,w_size)

    #input image
    plt.figure()
    plt.title("adaptive median")
    plt.subplot(121)
    plt.imshow(noisy_img)
    plt.gray()
    #output image
    plt.subplot(122)
    plt.imshow(output)
    plt.gray()
    plt.show()

'''
img1,var_n = myAdaptiveLNR(img,300,90,370,150,17)
#img2 = myAdaptiveMed(img,21)
plt.figure()
plt.imshow(img2)
#img2 = ndimage.filters.gaussian_filter(img,1)
#plt.figure()
#plt.imshow(img2)


Original Image
1065.217393601984


NameError: name 'img2' is not defined